In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
# input
idx_file="../gwas-current-gz-wc.20200629-131527.tsv"
wc_l_file='4_wc_l.tsv'

# output
map_file='6_map.tsv'


In [10]:
wc_l_df <- fread(wc_l_file) %>%
rename('wc_l'='#wc_l') %>%
separate(file, c(rep(NA, 1), 'population', 'basename'), sep='/', remove=F, fill='right', extra='drop') %>%
separate(basename, c(NA, 'GBE_ID', NA), sep='\\.', remove=T, fill='right', extra='drop')


In [14]:
wc_l_df %>%
count(population)

population,n
<chr>,<int>
others,89
related,91


In [15]:
idx_df <- fread(idx_file) %>%
rename('symlink'='#symlink')


In [22]:
idx_df %>%
filter(wc_l == 1080278) %>%
separate(symlink, c(rep(NA, 9), 'population', 'basename'), sep='/', remove=F, fill='right', extra='drop') %>%
separate(basename, c(NA, 'GBE_ID', NA), sep='\\.', remove=T, fill='right', extra='drop') %>%
rename('wc_l_main'='wc_l') %>%
left_join(
    wc_l_df %>% rename('wc_l_sub'='wc_l'),
    by=c('GBE_ID', 'population')
) %>%
select(GBE_ID, population, symlink, location, file, wc_l_main, wc_l_sub) -> check_df 


In [27]:
check_df %>% dim() %>% print()
check_df %>% filter(is.na(file)) %>% dim() %>% print()
check_df %>% drop_na(file) %>% dim() %>% print()

[1] 407   7
[1] 227   7
[1] 180   7


In [28]:
check_df %>%
drop_na(file) %>%
select(GBE_ID, population, symlink, location, file) %>%
rename('#GBE_ID' = 'GBE_ID') %>%
fwrite(map_file, sep='\t', na = "NA", quote=F)
